# A new IMG

<img src="https://images.seeklogo.com/logo-png/14/1/umbrella-corporation-residentevil-logo-png_seeklogo-144673.png" width="10%">

In [ ]:

# Hide inputs (you can toggle by setting SHOW_CODE=True and re-running this cell)
from IPython.display import HTML, display
SHOW_CODE = False
_style_id = "hide-code-style"

def _set_code_visibility(show: bool):
    css = "" if show else "div.jp-Cell-inputWrapper { display: none; }"
    html = f"<style id='{_style_id}'>{css}</style>"
    display(HTML(html))

_set_code_visibility(SHOW_CODE)



%pip install -q ipywidgets

# (Optional but recommended) warm-up big libs so the kernel loads them first
import numpy as _np, pandas as _pd, matplotlib.pyplot as _plt  # noqa: F401

# Import the module from the same folder as the notebook
from feature_app import main
main()
